In [1]:
import pandas as pd
from fhir.resources.encounter import Encounter
from fhir.resources.identifier import Identifier
from fhir.resources.period import Period
from fhir.resources.coding import Coding
from fhir.resources.codeableconcept import CodeableConcept
from datetime import datetime
import json
from fhir.resources.observation import ObservationComponent
from decimal import Decimal

#import simplejson as json
import math
import fhir.resources
print(fhir.resources.__version__)
df = pd.read_csv('sample_health_data_v2/observations.tsv', sep='\t')
df

6.5.0


,id,effective,patient_id,encounter_id,code,code_display,component_value,component_unit,component_type,category
0,obs_1,11/14/2014 01:48:13 PM,pat_9,enc_79,8302-2,Body Height,159.3000,cm,numeric,Vital signs
1,obs_2,11/14/2014 01:48:13 PM,pat_9,enc_79,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,2.0000,{score},numeric,Vital signs
2,obs_3,11/14/2014 01:48:13 PM,pat_9,enc_79,29463-7,Body Weight,44.0000,kg,numeric,Vital signs
3,obs_4,11/14/2014 01:48:13 PM,pat_9,enc_79,39156-5,Body mass index (BMI) [Ratio],17.3400,kg/m2,numeric,Vital signs
4,obs_5,11/14/2014 01:48:13 PM,pat_9,enc_79,59576-9,Body mass index (BMI) [Percentile] Per age and...,2.7588,%,numeric,Vital signs
...,...,...,...,...,...,...,...,...,...,...
748,obs_749,07/01/2014 02:50:16 AM,pat_3,enc_21,8867-4,Heart rate,65.0000,/min,numeric,Vital signs
749,obs_750,07/01/2014 02:50:16 AM,pat_3,enc_21,9279-1,Respiratory rate,13.0000,/min,numeric,Vital signs
750,obs_751,07/01/2014 02:50:16 AM,pat_3,enc_21,72166-2,Tobacco smoking status,NaN,NaN,NaN,Social history
751,obs_752,07/01/2014 03:39:48 AM,pat_3,enc_21,93025-5,Protocol for Responding to and Assessing Patie...,NaN,NaN,NaN,Survey


In [2]:
df["category"] = df["category"].replace(
    "Survey",
    "survey")
df["category"] = df["category"].replace(
    "Vital signs",
    "vital-signs")
df["category"] = df["category"].replace(
    "Social history",
    "social-history")
df["category"] = df["category"].replace(
    "Exam",
    "exam")
df["category"] = df["category"].replace(
    "Laboratory",
    "laboratory")
df["code_display"] = df["code_display"].replace(
    "Triglycerides",
    "Triglyceride [Mass/volume] in Serum or Plasma")
df["code_display"] = df["code_display"].replace(
    "Low Density Lipoprotein Cholesterol",
    "Cholesterol in LDL [Mass/volume] in Serum or Plasma by Direct assay")
df

,id,effective,patient_id,encounter_id,code,code_display,component_value,component_unit,component_type,category
0,obs_1,11/14/2014 01:48:13 PM,pat_9,enc_79,8302-2,Body Height,159.3000,cm,numeric,vital-signs
1,obs_2,11/14/2014 01:48:13 PM,pat_9,enc_79,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,2.0000,{score},numeric,vital-signs
2,obs_3,11/14/2014 01:48:13 PM,pat_9,enc_79,29463-7,Body Weight,44.0000,kg,numeric,vital-signs
3,obs_4,11/14/2014 01:48:13 PM,pat_9,enc_79,39156-5,Body mass index (BMI) [Ratio],17.3400,kg/m2,numeric,vital-signs
4,obs_5,11/14/2014 01:48:13 PM,pat_9,enc_79,59576-9,Body mass index (BMI) [Percentile] Per age and...,2.7588,%,numeric,vital-signs
...,...,...,...,...,...,...,...,...,...,...
748,obs_749,07/01/2014 02:50:16 AM,pat_3,enc_21,8867-4,Heart rate,65.0000,/min,numeric,vital-signs
749,obs_750,07/01/2014 02:50:16 AM,pat_3,enc_21,9279-1,Respiratory rate,13.0000,/min,numeric,vital-signs
750,obs_751,07/01/2014 02:50:16 AM,pat_3,enc_21,72166-2,Tobacco smoking status,NaN,NaN,NaN,social-history
751,obs_752,07/01/2014 03:39:48 AM,pat_3,enc_21,93025-5,Protocol for Responding to and Assessing Patie...,NaN,NaN,NaN,survey


In [3]:
df.to_csv("updatedobservation.tsv", sep='\t', index=False)

In [4]:
with open('test/observations.ndjson', 'r') as file:
    dataa = [json.loads(line) for line in file]

"""

     
{'resourceType': 'Observation',
  'identifier': [{'system': 'UPMC Observation Clincial Event ID',
    'value': 'obs_1'}],
  'status': 'none',
  'category': [{'coding': [{'system': 'https://snomed.info/sct',
      'code': 'Vital signs',
      'display': 'Vital signs'}]}],
  'code': {'coding': [{'system': 'http://loinc.org',
     'code': '8302-2',
     'display': 'Body Height'}]},
  'subject': {'reference': 'pat_9'},
  'encounter': {'reference': 'enc_79'},
  'effectiveDateTime': '2014-11-14T13:48:13-05:00',
  'component': [{'code': {'coding': [{'system': 'https://loinc.org',
       'code': '8302-2',
       'display': 'Body Height'}]},
    'valueQuantity': {'value': 159.3,
     'unit': 'cm',
     'system': 'https://unitsofmeasure.org',
     'code': 'cm'}}]},


{'resourceType': 'Observation', 
'identifier': [{'value': 'obs_1'}], 
'status': 'registered', 
'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category', 
'code': 'vital-signs', 
'display': 'Vital Signs'}]}], 
'code': {'coding': [{'system': 'http://loinc.org', 
'code': '8302-2', 
'display': 'Body Height'}]}, 
'subject': {'reference': 'pat_9'}, 
'encounter': {'reference': 'enc_79'}, 
'effectiveDateTime': '2014-11-14T13:48:13-05:00', 
'component': [{'code': {'coding': [{'system': 'http://loinc.org', 
'code': '8302-2', 
'display': 'Body Height'}]}, 
'valueQuantity': {'value': 159.3, 
'unit': 'cm', 
'system': 'http://unitsofmeasure.org', 
'code': 'cm'}}]}
"""
dataa[0:2]


[{'resourceType': 'Observation',
  'identifier': [{'system': 'UPMC Observation Clincial Event ID',
    'value': 'obs_1'}],
  'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category',
      'code': 'Vital signs',
      'display': 'Vital signs'}]}],
  'code': {'coding': [{'system': 'https://loinc.org',
     'code': '8302-2',
     'display': 'Body Height'}]},
  'subject': {'reference': 'pat_9'},
  'encounter': {'reference': 'enc_79'},
  'effectiveDateTime': '2014-11-14T13:48:13-05:00',
  'component': [{'code': {'coding': [{'system': 'https://loinc.org',
       'code': '8302-2',
       'display': 'Body Height'}]},
    'valueQuantity': {'value': 159.3,
     'unit': 'cm',
     'system': 'https://unitsofmeasure.org',
     'code': 'cm'}}]},
 {'resourceType': 'Observation',
  'identifier': [{'system': 'UPMC Observation Clincial Event ID',
    'value': 'obs_2'}],
  'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-categ

In [9]:
import pandas as pd
from fhir.resources.observation import Observation
from fhir.resources.period import Period
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding
from fhir.resources.reference import Reference
import json
from datetime import datetime
import re
from fhir.resources.identifier import Identifier
from datetime import datetime, timedelta, timezone
from fhir.resources.quantity import Quantity
import decimal

def convert_decimals_to_float(obj):
    """Recursively converts Decimal values in a dictionary or list to floats."""
    if isinstance(obj, dict):
        for key, value in obj.items():
            obj[key] = convert_decimals_to_float(value)
    elif isinstance(obj, list):
        for i in range(len(obj)):
            obj[i] = convert_decimals_to_float(obj[i])
    elif isinstance(obj, decimal.Decimal):  # Convert Decimal to float
        return float(obj)
    return obj

def parse_date(date_str):
    try:
        # Parse date and time from the format MM/DD/YYYY HH:MM:SS AM/PM
        parsed_date = datetime.strptime(date_str, "%m/%d/%Y %I:%M:%S %p")
        # Set the timezone offset to -05:00
        offset = timezone(timedelta(hours=-5))
        # Apply the timezone offset
        parsed_date_with_offset = parsed_date.replace(tzinfo=offset)
        # Convert to ISO format (YYYY-MM-DDTHH:MM:SS±HH:MM)
        return parsed_date_with_offset.isoformat()
    except ValueError:
        print(f"Invalid date format for '{date_str}'. Expected format: MM/DD/YYYY HH:MM:SS AM/PM.")
        return None
        
def convert_dates(record):
    if isinstance(record, dict):
        for key, value in record.items():
            if isinstance(value, datetime):  # If the value is a datetime object
                # Convert it to the desired format with timezone offset
                record[key] = convert_datetime_to_iso(value)
            elif isinstance(value, list):  # If the value is a list of items
                for item in value:
                    convert_dates(item)  # Recursively handle nested dictionaries
            else:
                convert_dates(value)  # Recursively handle nested dictionaries in the value
    elif isinstance(record, list):  # Handle lists outside of dicts
        for item in record:
            convert_dates(item)  # Recursively handle items in the list
    return record

def convert_datetime_to_iso(date_obj):
    # Assuming the input datetime is naive, and you want to apply a timezone of -05:00
    offset = timezone(timedelta(hours=-5))
    # Convert naive datetime to timezone-aware datetime
    if date_obj.tzinfo is None:
        date_obj = date_obj.replace(tzinfo=offset)
    # Return the ISO format with timezone offset
    return date_obj.isoformat()

def save_to_ndjson(data, output_file):
    with open(output_file, 'w') as f:
        for record in data:
            # Convert dates before writing
            record = convert_dates(record)
            record = convert_decimals_to_float(record)
            # Convert each OrderedDict to JSON and write it to the file
            f.write(json.dumps(record) + '\n')

def set_identifiers(observation, row):
    identifier = Identifier(system = 'UPMC Observation Clincial Event ID',
            value = row['id'])
    
    observation.identifier = [identifier]

def set_observation_subject(observation, row):
    subject=Reference(reference=f"{row['patient_id']}")
    observation.subject = subject
def set_observation_encounter(observation, row):
    subject=Reference(reference=f"{row['encounter_id']}")
    observation.encounter = subject
def set_observation_datetime(observation, row):
    start=parse_date(row['effective'])
    observation.effectiveDateTime = start
'''
def set_observation_component(observation, row):
    """Sets the component for an Observation resource."""
    
    if row['component_value'] is None or (isinstance(row['component_value'], float) and math.isnan(row['component_value'])):
        component = {
            "code": CodeableConcept(
                coding=[Coding(
                    system="https://loinc.org",
                    code=row['code'],
                    display=row['code_display']
                )]
            ),
        "dataAbsentReason": {
        			"coding": [
        				{
        					"system": "http://terminology.hl7.org/CodeSystem/data-absent-reason",
        					"code": "NaN",
        					"display": "Not a Number"
        				}
        			]
        		}
        }
    else:
        component = {
            "code": CodeableConcept(
                coding=[Coding(
                    system="https://loinc.org",
                    code=row['code'],
                    display=row['code_display']
                )]
            ),
            "valueQuantity": Quantity(
                value=row['component_value'],
                unit=row['component_unit'],
                system="https://unitsofmeasure.org",
                code=row['component_unit']
            )
        }
    #print(type(row['component_value']))
    # Add the component to the observation
    observation.component = [component]  
'''
def set_observation_component(observation, row):
    """Sets the component for an Observation resource."""
    
    # Always define the `code` field
    component_code = CodeableConcept(
        coding=[Coding(
            system="http://loinc.org",
            code=row['code'],
            display=row['code_display']
        )]
    )
    
    # Initialize the component
    component = ObservationComponent(code=component_code)
    
    # Handle `value[x]` or `dataAbsentReason`
    if row['component_value'] is None or (isinstance(row['component_value'], float) and math.isnan(row['component_value'])):
        component.dataAbsentReason = CodeableConcept(
            coding=[Coding(
                system="http://terminology.hl7.org/CodeSystem/data-absent-reason",
                code="not-a-number",  # Use the appropriate absent reason code
                display="Not a Number (NaN)"
            )]
        )
    else:
        component.valueQuantity = Quantity(
            value=Decimal(str(row['component_value'])),  # Ensure Decimal conversion
            unit=row.get('component_unit', ''),  # Use empty string as fallback for unit
            system="http://unitsofmeasure.org",
            code=row.get('component_unit', '')  # Use empty string as fallback for unit code
        )
    
    # Add the component to the observation
    observation.component = [component]  
def set_observation_category(observation,row):
    # Create the Coding object for the reason
    coding = CodeableConcept(
        coding = [Coding(
        system='http://terminology.hl7.org/CodeSystem/observation-category',  # SNOMED system for reason codes
        code=row['category'],  # Reason code (e.g., SNOMED code)
        display=row['category']  # Display name for the reason (e.g., diagnosis)
    )])
    
    
    # Set the reasonCode attribute in the encounter resource
    observation.category = [coding]  # Assign the EncounterReason to reasonCode
def create_observation_code(row, system="https://loinc.org", code="29463-7", display="Body weight"):
    """Creates a CodeableConcept for the Observation code."""
    return CodeableConcept(
        coding=[Coding(system=system, code=row['code'], display=row['code_display'])]
    )
def tsv_to_fhir_observations(tsv_file):
    df = pd.read_csv(tsv_file, sep='\t')
    observations = []

    for _, row in df.iterrows():
        observation = Observation(resourceType="Observation", status="none", code=create_observation_code(row))
        set_identifiers(observation, row)
        set_observation_component(observation, row)
        set_observation_datetime(observation, row)
        set_observation_subject(observation, row)
        set_observation_encounter(observation, row)
        set_observation_category(observation, row)
        observations.append(observation.dict())
    
    return observations
# Example usage
tsv_file = 'sample_health_data_v2/observations.tsv'
output_file = 'observations.ndjson'
observations = tsv_to_fhir_observations(tsv_file)
'''
def print_types(data, level=0):
    """Recursively prints the type of each value in a dictionary or list."""
    indent = "  " * level  # Indentation for nested structures
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{indent}{key}: {type(value).__name__}")
            print_types(value, level + 1)  # Recursive call for nested dict
    elif isinstance(data, list):
        for index, item in enumerate(data):
            print(f"{indent}[{index}]: {type(item).__name__}")
            print_types(item, level + 1)  # Recursive call for nested list
    else:
        print(f"{indent}{data}: {type(data).__name__}")

# Usage
# Assuming `observation` is your Observation resource object
print("Types of each value in Observation resources:")

print_types(observations[0]) 
'''
# Save to NDJSON
save_to_ndjson(observations, output_file)

In [10]:
with open('observations.ndjson', 'r') as file:
    data = [json.loads(line) for line in file]

# `data` is now a list of dictionaries (one for each JSON object in the file)
data[0:9]

[{'resourceType': 'Observation',
  'identifier': [{'system': 'UPMC Observation Clincial Event ID',
    'value': 'obs_1'}],
  'status': 'none',
  'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/observation-category',
      'code': 'Vital signs',
      'display': 'Vital signs'}]}],
  'code': {'coding': [{'system': 'https://loinc.org',
     'code': '8302-2',
     'display': 'Body Height'}]},
  'subject': {'reference': 'pat_9'},
  'encounter': {'reference': 'enc_79'},
  'effectiveDateTime': '2014-11-14T13:48:13-05:00',
  'component': [{'code': {'coding': [{'system': 'http://loinc.org',
       'code': '8302-2',
       'display': 'Body Height'}]},
    'valueQuantity': {'value': 159.3,
     'unit': 'cm',
     'system': 'http://unitsofmeasure.org',
     'code': 'cm'}}]},
 {'resourceType': 'Observation',
  'identifier': [{'system': 'UPMC Observation Clincial Event ID',
    'value': 'obs_2'}],
  'status': 'none',
  'category': [{'coding': [{'system': 'http://terminolog

In [8]:
import json

def compare_dicts(dict1, dict2, path=""):
    """Recursively compare two dictionaries and return a list of differences."""
    differences = []
    
    # Get all keys in both dictionaries
    all_keys = set(dict1.keys()).union(set(dict2.keys()))
    
    for key in all_keys:
        # Full path for the current key
        current_path = f"{path}.{key}" if path else key

        if key not in dict1:
            differences.append({"path": current_path, "file1": None, "file2": dict2[key]})
        elif key not in dict2:
            differences.append({"path": current_path, "file1": dict1[key], "file2": None})
        else:
            # Check if the values are both dictionaries for nested comparison
            if isinstance(dict1[key], dict) and isinstance(dict2[key], dict):
                differences.extend(compare_dicts(dict1[key], dict2[key], path=current_path))
            # Check if the values are different
            elif dict1[key] != dict2[key]:
                differences.append({"path": current_path, "file1": dict1[key], "file2": dict2[key]})
    
    return differences

def compare_ndjson_files(file1, file2):
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        lines1 = f1.readlines()
        lines2 = f2.readlines()

        # Check if the number of lines match
        if len(lines1) != len(lines2):
            return False, "The NDJSON files have a different number of lines."

        all_differences = []

        # Compare line by line
        for i, (line1, line2) in enumerate(zip(lines1, lines2)):
            json_obj1 = json.loads(line1)
            json_obj2 = json.loads(line2)

            # Pop out 'status' from json_obj1 before comparison
            json_obj1.pop('status', None)

            # Compare the JSON objects
            differences = compare_dicts(json_obj1, json_obj2)
            if differences:
                all_differences.append({
                    "line": i + 1,  # Line number (1-based index)
                    "differences": differences
                })

        return True if not all_differences else False, all_differences

def print_differences(differences):
    """Print the differences in a readable format."""
    if not differences:
        print("The NDJSON files are the same.")
    else:
        for diff in differences:
            print(f"Difference found in line {diff['line']}:")
            for item in diff["differences"]:
                print(f"- Path: {item['path']}")
                print(f"  File 1: {json.dumps(item['file1'], indent=2)}")
                print(f"  File 2: {json.dumps(item['file2'], indent=2)}")
                print("-" * 40)

# Example usage
file1 = 'observations.ndjson'
file2 = 'test/observations.ndjson'

are_same, differences = compare_ndjson_files(file1, file2)

if are_same:
    print("The NDJSON files are the same.")
else:
    print("The NDJSON files are different:")
    print_differences(differences)


The NDJSON files are different:
Difference found in line 11:
- Path: effectiveDateTime
  File 1: "2017-07-04T02:50:16-05:00"
  File 2: "2017-07-04T02:50:16-04:00"
----------------------------------------
Difference found in line 12:
- Path: effectiveDateTime
  File 1: "2017-07-04T02:50:16-05:00"
  File 2: "2017-07-04T02:50:16-04:00"
----------------------------------------
Difference found in line 13:
- Path: effectiveDateTime
  File 1: "2017-07-04T02:50:16-05:00"
  File 2: "2017-07-04T02:50:16-04:00"
----------------------------------------
Difference found in line 14:
- Path: effectiveDateTime
  File 1: "2017-07-04T02:50:16-05:00"
  File 2: "2017-07-04T02:50:16-04:00"
----------------------------------------
Difference found in line 15:
- Path: effectiveDateTime
  File 1: "2017-07-04T02:50:16-05:00"
  File 2: "2017-07-04T02:50:16-04:00"
----------------------------------------
Difference found in line 16:
- Path: effectiveDateTime
  File 1: "2017-07-04T02:50:16-05:00"
  File 2: "2017